# EDA

In [45]:
import os
import ast
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas import json_normalize

## 전체 데이터 파악

In [90]:
train_path = '/Volumes/Expansion/google_analytics_customer/train_v2.csv'
test_path = '/Volumes/Expansion/google_analytics_customer/test_v2.csv'

In [4]:
df_all = pd.read_csv(train_path)

df_all.head()

/var/folders/l3/cq8y8n3s03v315zt58m2zrv80000gn/T/ipykernel_53138/853162711.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(train_path)


,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


- 컬럼 확인

In [56]:
print(df_all.columns)

Index(['channelGrouping', 'customDimensions', 'date', 'device',
       'fullVisitorId', 'geoNetwork', 'hits', 'socialEngagementType', 'totals',
       'trafficSource', 'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')


- 데이터셋 개요

In [6]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 13 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   channelGrouping       object
 1   customDimensions      object
 2   date                  int64 
 3   device                object
 4   fullVisitorId         object
 5   geoNetwork            object
 6   hits                  object
 7   socialEngagementType  object
 8   totals                object
 9   trafficSource         object
 10  visitId               int64 
 11  visitNumber           int64 
 12  visitStartTime        int64 
dtypes: int64(4), object(9)
memory usage: 169.4+ MB


### nested data를 값으로 가지는 컬럼 확인

- "[{key : value}]" 형태의 값을 가지는 hits 컬럼 확인

In [43]:
print(type(df_all['hits'][0]))
df_all['hits'][0]

<class 'str'>


"[{'hitNumber': '1', 'time': '0', 'hour': '17', 'minute': '0', 'isInteraction': True, 'isEntrance': True, 'isExit': True, 'referer': 'https://www.google.co.uk/search?q=water+bottle&ie=utf-8&num=100&oe=utf-8&hl=en&gl=GB&uule=w+CAIQIFISCamRx0IRO1oCEXoliDJDoPjE&glp=1&gws_rd=cr&fg=1', 'page': {'pagePath': '/google+redesign/bags/water+bottles+and+tumblers', 'hostname': 'shop.googlemerchandisestore.com', 'pageTitle': 'Water Bottles & Tumblers | Drinkware | Google Merchandise Store', 'pagePathLevel1': '/google+redesign/', 'pagePathLevel2': '/bags/', 'pagePathLevel3': '/water+bottles+and+tumblers', 'pagePathLevel4': ''}, 'transaction': {'currencyCode': 'USD'}, 'item': {'currencyCode': 'USD'}, 'appInfo': {'screenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers', 'landingScreenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers', 'exitScreenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tum

In [ ]:
s = df_all['hits'][0]   # 첫번째 행 값 확인
lst = ast.literal_eval(s)

lst[0]

{'hitNumber': '1',
 'time': '0',
 'hour': '17',
 'minute': '0',
 'isInteraction': True,
 'isEntrance': True,
 'isExit': True,
 'referer': 'https://www.google.co.uk/search?q=water+bottle&ie=utf-8&num=100&oe=utf-8&hl=en&gl=GB&uule=w+CAIQIFISCamRx0IRO1oCEXoliDJDoPjE&glp=1&gws_rd=cr&fg=1',
 'page': {'pagePath': '/google+redesign/bags/water+bottles+and+tumblers',
  'hostname': 'shop.googlemerchandisestore.com',
  'pageTitle': 'Water Bottles & Tumblers | Drinkware | Google Merchandise Store',
  'pagePathLevel1': '/google+redesign/',
  'pagePathLevel2': '/bags/',
  'pagePathLevel3': '/water+bottles+and+tumblers',
  'pagePathLevel4': ''},
 'transaction': {'currencyCode': 'USD'},
 'item': {'currencyCode': 'USD'},
 'appInfo': {'screenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers',
  'landingScreenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers',
  'exitScreenName': 'shop.googlemerchandisestore.com/google+redesign/b

- "{key : value}" 형태의 값을 가지는 device 컬럼 확인

In [8]:
print(type(df_all['device'][0]))
df_all['device'][0]

<class 'str'>


'{"browser": "Firefox", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}'

In [ ]:
s = df_all['device'][0]    # 첫번째 행 값 확인

data = json.loads(s)

data

{'browser': 'Firefox',
 'browserVersion': 'not available in demo dataset',
 'browserSize': 'not available in demo dataset',
 'operatingSystem': 'Windows',
 'operatingSystemVersion': 'not available in demo dataset',
 'isMobile': False,
 'mobileDeviceBranding': 'not available in demo dataset',
 'mobileDeviceModel': 'not available in demo dataset',
 'mobileInputSelector': 'not available in demo dataset',
 'mobileDeviceInfo': 'not available in demo dataset',
 'mobileDeviceMarketingName': 'not available in demo dataset',
 'flashVersion': 'not available in demo dataset',
 'language': 'not available in demo dataset',
 'screenColors': 'not available in demo dataset',
 'screenResolution': 'not available in demo dataset',
 'deviceCategory': 'desktop'}

In [94]:
df_all['totals'][60000]

'{"visits": "1", "hits": "16", "pageviews": "14", "timeOnSite": "620", "transactions": "1", "transactionRevenue": "27980000", "totalTransactionRevenue": "35980000", "sessionQualityDim": "96"}'

### test code를 위한 상위 5개 행만 저장

In [59]:
sample_df = df_all.head()

sample_df.to_csv('sample_df.csv')

In [ ]:
columns = ['device', 'geoNetwork', 'totals', 'trafficSource'] # Columns that have json format

#Code to transform the json format columns in table
def json_read(df):
    #joining the [ path + df received]
    
    #Importing the dataset
    df = pd.read_csv(df, 
                     converters={column: json.loads for column in columns}, # loading the json columns properly
                     dtype={'fullVisitorId': 'str'}) # transforming this column to string
    
    for column in columns: #loop to finally transform the columns in data frame
        #It will normalize and set the json to a table
        column_as_df = json_normalize(df[column]) 
        # here will be set the name using the category and subcategory of json columns
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        # after extracting the values, let drop the original columns
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    # Printing the shape of dataframes that was imported     
    print(f"Loaded {os.path.basename(df)}. Shape: {df.shape}")
    return df # returning the df after importing and transforming

In [80]:
columns = ['device', 'geoNetwork', 'totals', 'trafficSource']   # JSON이 있는 컬럼

def json_read(file_path):
    df = pd.read_csv(file_path, 
                     converters={column: json.loads for column in columns},
                     dtype={'fullVisitorId': 'str'})
    
    # JSON 컬럼 처리
    for column in columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    print(f"Loaded {file_path}. Shape: {df.shape}")
    return df

In [81]:
df_sample = json_read('/Users/taeheon/customer/sample_df.csv')

df_sample.head()

Loaded /Users/taeheon/customer/sample_df.csv. Shape: (5, 51)


,Unnamed: 0,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,...,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect
0,0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,...,1,1,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN
1,1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,...,NaN,2,28,(not set),sites.google.com,referral,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN
2,2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,...,1,1,38,(not set),(direct),(none),NaN,not available in demo dataset,NaN,True
3,3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,9075655783635761930,"[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,1508169851,1,1508169851,...,1,1,1,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN
4,4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,6960673291025684308,"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,1508190552,1,1508190552,...,1,1,52,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN


In [83]:
df_sample.columns, len(df_sample.columns)

(Index(['Unnamed: 0', 'channelGrouping', 'customDimensions', 'date',
        'fullVisitorId', 'hits', 'socialEngagementType', 'visitId',
        'visitNumber', 'visitStartTime', 'device.browser',
        'device.browserVersion', 'device.browserSize', 'device.operatingSystem',
        'device.operatingSystemVersion', 'device.isMobile',
        'device.mobileDeviceBranding', 'device.mobileDeviceModel',
        'device.mobileInputSelector', 'device.mobileDeviceInfo',
        'device.mobileDeviceMarketingName', 'device.flashVersion',
        'device.language', 'device.screenColors', 'device.screenResolution',
        'device.deviceCategory', 'geoNetwork.continent',
        'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region',
        'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.cityId',
        'geoNetwork.networkDomain', 'geoNetwork.latitude',
        'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits',
        'totals.hits', 'totals.pageviews', 'to

In [ ]:
df_sample['totals.t']